In [ ]:
import pickle, os
import requests as reqlib

import subprocess
import time

import pandas as pd
import numpy as np
import cma

In [48]:
reference_path   = "../../results_aurore/transit/reference_MZ_sample.parquet"
routing_endpoint = "http://localhost:8054/router/transit"

output_path = "../../results_aurore/transit/calibration.p"

# Objective is observation_based or distribution_based
objective = "distribution_based"

In [49]:
df_reference = pd.read_parquet(reference_path)

In [50]:
# Identify modes and maximum transfers
modes = [c.replace("legs_", "") for c in df_reference.columns if c.startswith("legs_")]
maximum_transfers = df_reference["transfers"].max()

In [51]:
# Convert to requests
requests = []

df_reference["request_index"] = np.arange(len(df_reference))

for index, row in df_reference.iterrows():
    requests.append({
        "request_index": int(row["request_index"]),
        "origin_x": row["origin_x"],
        "origin_y": row["origin_y"],
        "destination_x": row["destination_x"],
        "destination_y": row["destination_y"],
        "departure_time_s": row["departure_time"]
    })

In [52]:
# We need to be careful about the request/response size, so we send individual batches
maximum_batch_size = 4000

def query_endpoint_batched(requests, utilities):
    df_response = []
    batch_index = 0

    while batch_index * maximum_batch_size < len(requests):
        df_response.append(query_endpoint(
            requests[batch_index * maximum_batch_size : (batch_index + 1) * maximum_batch_size],
            utilities))
        
        batch_index += 1
    
    return pd.concat(df_response)

In [ ]:
# Define calibration variables
# Using values from the Zurich mode choice model, scaled so that rail TT utility is used as a reference fixed to -1.0
# The tram TT utility is absent from the MCM, using rail TT as a reference
variables = [
    { "name": "rail_u_h", "initial": -1.0, "fixed": True },
    { "name": "bus_u_h", "initial": -1.72 },
    { "name": "tram_u_h", "initial": -1.0 },
    { "name": "subway_u_h", "copy": "tram_u_h" },
    { "name": "other_u_h", "copy": "bus_u_h" },
    { "name": "wait_u_h", "initial": -1.72},
    { "name": "walk_u_h", "initial": -1.97 },
    { "name": "transfer_u", "initial": -0.39 }
]

In [54]:
# Extend with index information for CMA-ES evaluation
variables_map = { v["name"]: v for v in variables }

active_index = 0

# First find active variables
for variable in variables:
    if "fixed" in variable or "copy" in variable: 
        continue

    variables_map[variable["name"]] = variable
    
    variable["index"] = active_index
    variable["optimized"] = True
    active_index += 1

# Then treat fixed variables and those copying from others
for variable in variables:
    if "fixed" in variable:
        variable["index"] = None
    
    if "copy" in variable:
        assert not "initial" in variable
        variable["initial"] = variables_map[variable["copy"]]["initial"]
        variable["index"] = variables_map[variable["copy"]]["index"]

In [55]:
# Define the optimization objective
modes_weight = 1.0
transfers_weight = 1.0

def calculate_objective_observation_based(df_evaluation):
    df_evaluation = pd.merge(df_reference, df_evaluation, on = "request_index", 
        suffixes = ["_reference", "_evaluation"])
    
    df_evaluation["offset"] = transfers_weight * np.abs(
        df_evaluation["transfers_reference"] - df_evaluation["transfers_evaluation"])

    for mode in modes:
        df_evaluation["offset"] += modes_weight * np.abs(
            df_evaluation["legs_{}_reference".format(mode)] - df_evaluation["legs_{}_evaluation".format(mode)]
        )

    return np.sum(df_evaluation["offset"] * df_evaluation["weight"]) / df_evaluation["weight"].sum()

def calculate_objective_distribution_based(df_evaluation):
    df_evaluation = pd.merge(df_reference, df_evaluation, on = "request_index", 
        suffixes = ["_reference", "_evaluation"])
    
    reference_mode_distribution = []
    evaluation_mode_distribution = []

    for mode in modes:
        reference_mode_distribution.append((df_evaluation["legs_{}_reference".format(mode)] * df_evaluation["weight"]).sum())
        evaluation_mode_distribution.append((df_evaluation["legs_{}_evaluation".format(mode)] * df_evaluation["weight"]).sum())

    reference_mode_distribution = np.array(reference_mode_distribution) / np.sum(reference_mode_distribution)
    evaluation_mode_distribution = np.array(evaluation_mode_distribution) / np.sum(evaluation_mode_distribution)

    reference_transfer_distribution = []
    evaluation_transfer_distribution = []

    for transfers in range(maximum_transfers + 1):
        f_reference = df_evaluation["transfers_reference"] == transfers
        reference_transfer_distribution.append(df_evaluation.loc[f_reference, "weight"].sum())

        f_evaluation = df_evaluation["transfers_evaluation"] == transfers
        evaluation_transfer_distribution.append(df_evaluation.loc[f_evaluation, "weight"].sum())

    reference_transfer_distribution = np.array(reference_transfer_distribution) / np.sum(reference_transfer_distribution)
    evaluation_transfer_distribution = np.array(evaluation_transfer_distribution) / np.sum(evaluation_transfer_distribution)

    mode_distribution_offset = np.abs(reference_mode_distribution - evaluation_mode_distribution)
    transfer_distribution_offset = np.abs(reference_transfer_distribution - evaluation_transfer_distribution)

    return transfers_weight * np.sum(transfer_distribution_offset) + modes_weight * np.sum(mode_distribution_offset)

In [56]:
# Prepare function to convert CMA-ES' candidate to utilities
def prepare_utilities(values):
    utilities = {}
    
    for variable in variables:
        if variable["index"] is not None:
            utilities[variable["name"]] = values[variable["index"]]
        else:
            utilities[variable["name"]] = variable["initial"]
    
    return utilities

In [57]:
# Prepare bounds and initial values
initial = []
bounds = [[], []]

for variable in variables:
    if "optimized" in variable:
        initial.append(variable["initial"])
        bounds[0].append(-np.inf)
        bounds[1].append(0.0)

In [58]:
# Prepare querying the routing server
def query_endpoint(requests, utilities):
    response = reqlib.post(routing_endpoint, json = {
        "batch": requests,
        "utilities": utilities
    })

    assert response.status_code == 200

    df_response = { 
        "request_index": [],
        "transfers": []
    }

    modes_and_subway = modes.copy()
    modes_and_subway.append("subway")

    for mode in modes:
        df_response["legs_{}".format(mode)] = []

    for row in response.json():
        df_response["request_index"].append(row["request_index"])
        df_response["transfers"].append(row["transfers"])
        
        for mode in modes_and_subway:
            if mode in modes:
                if mode in row["vehicle_legs_by_mode"]:
                    df_response["legs_{}".format(mode)].append(row["vehicle_legs_by_mode"][mode])
                else:
                    df_response["legs_{}".format(mode)].append(0)
            else:
                if mode in row["vehicle_legs_by_mode"]:
                    df_response["legs_tram"][-1] += row["vehicle_legs_by_mode"][mode]

    df_response = pd.DataFrame(df_response)
    return df_response

In [59]:
# Test connection
assert len(query_endpoint(requests[:5], prepare_utilities(initial))) == 5

In [60]:
# Configure CMA-ES
seed = 1000
sigma = 0.5
iterations = 20

options = cma.CMAOptions()
options.set("bounds", bounds)
options.set("seed", seed)

algorithm = cma.CMAEvolutionStrategy(initial, sigma, options)

# Load cached data for previous iterations
history = []

if os.path.exists(output_path):
    with open(output_path, "rb") as f:
        history = pickle.load(f)

        algorithm.feed_for_resume(
            [h["candidate"] for h in history[1:]], # first one is initial
            [h["objective"] for h in history[1:]]
        )

# Choose objective
if objective == "observation_based":
    calculate_objective = calculate_objective_observation_based
elif objective == "distribution_based":
    calculate_objective = calculate_objective_distribution_based
else:
    raise RuntimeError("Unknown objective")

# Perform a new batch of iterations
for iteration in range(iterations):
    initial_evaluation = len(history) == 0
    candidates = [initial]

    if not initial_evaluation:
        candidates = algorithm.ask()

    objectives = []

    for candidate in candidates:
        utilities = prepare_utilities(candidate)
        df_response = query_endpoint_batched(requests, utilities)
        objective = calculate_objective(df_response)

        objectives.append(objective)

        history.append({
            "candidate": candidate,
            "utilities": utilities,
            "objective": objective,
            "evaluation": df_response,
            "initial": initial_evaluation
        })

    if not initial_evaluation:
        algorithm.tell(candidates, objectives)
        algorithm.disp()

    # Save after a successful CMA-ES iteration
    with open(output_path, "wb+") as f:
        pickle.dump(history, f)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 5 (seed=1000, Fri May 30 15:15:38 2025)
   24    192 2.265136249868759e-01 2.4e+00 1.68e-01  1e-01  1e-01 0:29.2
   25    200 2.239266420732613e-01 2.2e+00 1.77e-01  1e-01  2e-01 1:02.8
   26    208 2.210489054296755e-01 2.6e+00 1.60e-01  9e-02  1e-01 1:45.6
   27    216 1.242452594134871e-01 2.6e+00 1.48e-01  8e-02  1e-01 2:08.6
   28    224 1.754099248196890e-01 2.7e+00 1.44e-01  8e-02  1e-01 2:26.8


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))